# Usage

The following pages go through the usual workflow when using {mod}`tensorwaves`. The output in each of these steps is stored, so that they can be run separately. This page shows some of the highlights of the complete workflow.

```{toctree}
---
maxdepth: 2
---
usage/1_create_model
usage/2_generate_data
usage/3_perform_fit
```

## Create an amplitude model

:::{seealso}

{doc}`usage/1_create_model`

:::

In [ ]:
import expertsystem as es
import graphviz
import matplotlib.pyplot as plt
import pandas as pd
from expertsystem.amplitude.dynamics import set_resonance_dynamics
from expertsystem.amplitude.dynamics.builder import (
    create_relativistic_breit_wigner_with_ff,
)
from tensorwaves.data.generate import generate_data, generate_phsp
from tensorwaves.estimator import SympyUnbinnedNLL
from tensorwaves.optimizer.callbacks import CSVSummary
from tensorwaves.optimizer.minuit import Minuit2
from tensorwaves.physics.amplitude import Intensity, SympyModel
from tensorwaves.physics.helicity_formalism.kinematics import (
    HelicityKinematics,
    ParticleReactionKinematicsInfo,
    SubSystem,
)

In [ ]:
result = es.generate_transitions(
    initial_state=("J/psi(1S)", [-1, +1]),
    final_state=["gamma", "pi0", "pi0"],
    allowed_intermediate_particles=["f(0)"],
    allowed_interaction_types=["strong", "em"],
    formalism_type="canonical-helicity",
)
graphs = result.collapse_graphs()
dot = es.io.convert_to_dot(graphs)
graphviz.Source(dot)

In [ ]:
model = es.generate_amplitudes(result)
model.expression.top

In [ ]:
for name in result.get_intermediate_particles().names:
    set_resonance_dynamics(
        model, name, create_relativistic_breit_wigner_with_ff
    )

In [ ]:
display(*model.expression.dynamics)

In [ ]:
display(*sorted(model.parameters, key=lambda s: s.name))

In [ ]:
list(model.expression.dynamics.values())[1]

## Generate toy MC sample

:::{seealso}

{doc}`usage/2_generate_data`

:::

In [ ]:
sympy_model = SympyModel(
    expression=model.expression.full_expression,
    parameters={k: v.value for k, v in model.parameters.items()},
    variables={},
)
intensity = Intensity(sympy_model)

In [ ]:
for state_id, particle in model.kinematics.final_state.items():
    print(f"ID {state_id}:", particle.name)

In [ ]:
kin = HelicityKinematics(
    ParticleReactionKinematicsInfo(
        initial_state_names=[
            x.name for x in model.kinematics.initial_state.values()
        ],
        final_state_names=[
            x.name for x in model.kinematics.final_state.values()
        ],
        particles=model.particles,
        fs_id_event_pos_mapping=dict(
            {k: i for i, k in enumerate(model.kinematics.final_state.keys())}
        ),
    )
)
kin.register_subsystem(SubSystem([[3, 4], [2]], [], []))
kin.register_subsystem(SubSystem([[3], [4]], [2], []))
kin.register_invariant_mass([2, 4]);

In [ ]:
phsp_sample = generate_phsp(300_000, kin)
data_sample = generate_data(30_000, kin, intensity)

In [ ]:
phsp_set = kin.convert(phsp_sample)
data_set = kin.convert(data_sample)

In [ ]:
import numpy as np
from matplotlib import cm

intermediate_states = sorted(
    (
        p
        for p in model.particles
        if p not in model.kinematics.final_state.values()
        and p not in model.kinematics.initial_state.values()
    ),
    key=lambda p: p.mass,
)

evenly_spaced_interval = np.linspace(0, 1, len(intermediate_states))
colors = [cm.rainbow(x) for x in evenly_spaced_interval]


def indicate_masses():
    plt.xlabel("$m$ [GeV]")
    for i, p in enumerate(intermediate_states):
        plt.gca().axvline(
            x=p.mass, linestyle="dotted", label=p.name, color=colors[i]
        )

In [ ]:
data_frame = pd.DataFrame(data_set)
data_frame["m_3+4"].hist(bins=100, alpha=0.5, density=True)
indicate_masses()
plt.legend();

# Optimize the amplitude model

:::{seealso}

{doc}`usage/3_perform_fit`

:::

In [ ]:
estimator = SympyUnbinnedNLL(sympy_model, data_set, phsp_set, backend="jax")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def compare_model(
    variable_name,
    data_set,
    phsp_set,
    intensity_model,
    bins=150,
):
    data = data_set[variable_name]
    phsp = phsp_set[variable_name]
    intensities = intensity_model(phsp_set)
    plt.hist(data, bins=bins, alpha=0.5, label="data", density=True)
    plt.hist(
        phsp,
        weights=intensities,
        bins=bins,
        histtype="step",
        color="red",
        label="initial fit model",
        density=True,
    )
    indicate_masses()
    plt.legend()

In [ ]:
initial_parameters = {
    "C[J/\\psi(1S) \\to f_{0}(1500)_{0} \\gamma_{+1};f_{0}(1500) \\to \\pi^{0}_{0} \\pi^{0}_{0}]": 1.0
    + 0.0j,
    "Gamma_f(0)(500)": 0.3,
    "Gamma_f(0)(980)": 0.1,
    "m_f(0)(1710)": 1.75,
    "Gamma_f(0)(1710)": 0.2,
}
intensity.update_parameters(initial_parameters)
compare_model("m_3+4", data_set, phsp_set, intensity)
print("Number of free parameters:", len(initial_parameters))

```{tip}
Insert behavior into the {class}`.Optimizer` by defining a custom {class}`.Callback` class. Here's one that live updates a plot of the latest fit model!
```

In [ ]:
import os

from IPython.display import clear_output
from tensorwaves.optimizer.callbacks import Callback, CallbackList


class PyplotCallback(Callback):
    def __init__(self, step_size=10):
        self.__step_size = step_size
        self.__fig, self.__ax = plt.subplots(1, figsize=(8, 5))
        self.__latest_parameters = {}

    def on_iteration_end(self, function_call, logs=None):
        self.__latest_parameters = logs["parameters"]
        if function_call % self.__step_size != 0:
            return
        if "READTHEDOCS" in os.environ:
            return
        self.update_plot()
        clear_output(wait=True)
        display(plt.gcf())

    def on_function_call_end(self):
        self.update_plot()

    def update_plot(self):
        bins = 100
        data = data_set["m_3+4"]
        phsp = phsp_set["m_3+4"]
        intensity.update_parameters(self.__latest_parameters)
        intensities = intensity(phsp_set)
        self.__ax.cla()
        self.__ax.hist(data, bins=bins, alpha=0.5, label="data", density=True)
        self.__ax.hist(
            phsp,
            weights=intensities,
            bins=bins,
            histtype="step",
            color="red",
            label="fit model",
            density=True,
        )
        self.__ax.set_xlim((0.25, 2.5))
        self.__ax.set_ylim((0, 1.9))
        indicate_masses()
        plt.gcf().legend()

In [ ]:
minuit2 = Minuit2(
    callback=CallbackList(
        [CSVSummary("traceback.csv", step_size=2), PyplotCallback()]
    )
)
fit_result = minuit2.optimize(estimator, initial_parameters)
fit_result

In [ ]:
fit_traceback = pd.read_csv("traceback.csv")
fig, (ax1, ax2) = plt.subplots(
    2, figsize=(7, 9), sharex=True, gridspec_kw={"height_ratios": [1, 2]}
)
fit_traceback.plot("function_call", "estimator_value", ax=ax1)
fit_traceback.plot("function_call", list(initial_parameters), ax=ax2)
fig.tight_layout()
ax2.set_xlabel("function call");